In [32]:
import io
import os
import pandas as pd
import google.generativeai as genai

In [46]:
genai.configure(api_key="Cho api key của mọi người vào đây")
model = genai.GenerativeModel('gemini-pro')

def convert_excel_to_csv(excel_file_path, csv_dir_path):
    if not os.path.exists(csv_dir_path):
        os.makedirs(csv_dir_path)
    
    xls = pd.ExcelFile(excel_file_path)
    
    for sheet in xls.sheet_names:
        path = os.path.join(csv_dir_path, f"{sheet}.csv")
        data_xls = pd.read_excel(excel_file_path, sheet, index_col=None)
        data_xls.to_csv(path, encoding='utf-8', index=False)

def convert_csv_to_excel(excel_file_path, csv_file_path):
    if not os.path.exists(excel_file_path):
        pd.DataFrame().to_excel(excel_file_path)
    
    data_csv = pd.read_csv(csv_file_path)
    data_csv.to_excel(excel_file_path, index=False)

def read_csv_pandas(file_path):
    df = pd.read_csv(file_path)
    return df

def create_prompt_pandas(df_path, chose, user_chose):
    df_text = {}
    for file in os.listdir(df_path):
        if file.endswith(".csv"):
            df = read_csv_pandas(os.path.join(df_path, file))
            df_text[file.split(".")[0]] = df.to_csv(index=False)
    prompt = ""

    key = "Quanlydonhang"
    search_key = "Hangtonkho"
    info_user_prompt = "Tên tôi là Trần Lê Anh và số điện thoại là 0987654321."
    if chose[user_chose] == "Nhập thông tin":
        prompt = f"""
			Dữ liệu CSV:
			{df_text[key]}
            Dữ liệu người dùng:
            {info_user_prompt}
			Hãy thêm dữ liệu trong dữ liệu người dùng vào trong dữ liệu CSV 
			(Chỉ thêm những thông tin tồn tại trong dữ liệu người dùng vào trong dữ liệu người dùng).
			Trả về kết quả là một CSV mới (bao gồm cả header và data).
		"""
    elif chose[user_chose] == "Tạo đơn hàng":
        
        order_prompt = "Tôi muốn mua 2 quyển sách Tôi tài giỏi, bạn cũng thể"
        search_prompt = f"""
            Dữ liệu CSV:
            {df_text[search_key]}
            Tìm kiếm thông tin {order_prompt} có trong kho
        """
        
        gemini_response = get_gemini_response(search_prompt)
        
        prompt = f"""
            Dữ liệu CSV:
            {df_text[key]}
            Hãy thêm dữ liệu trong {order_prompt} và {gemini_response} vào trong dữ liệu CSV với thông tin người dùng {info_user_prompt}
            (Cập nhât thông tin trong cột ID của dữ liệu CSV thành thông tin ID cuốn sách).
            Trả về kết quả là một CSV mới (bao gồm cả header và data).
        """
    return prompt

def get_gemini_response(prompt):
    response = model.generate_content(prompt)
    return response.text

def write_csv_pandas(response_text, output_file_path):
    df = pd.read_csv(io.StringIO(response_text))
    df.to_csv(output_file_path, index=False)


if __name__ == '__main__':
    csv_file_path = 'CSV' # Tạo folder CSV chứa các file CSV ứng với các sheet trong file Excel
    output_file = 'New_Quanlydonhang.csv'
    
    chose = {
      	1 : "Nhập thông tin",
      	2 : "Tạo đơn hàng",
    } # Tạo trường hợp cho người dùng chọn thao tác
    
    convert_excel_to_csv("Products.xlsx", "CSV") # Chuyển file Excel thành file CSV
    # convert_csv_to_excel("New_Customer.xlsx", "CSV/Quanlydonhang.csv")
    
    prompt = create_prompt_pandas(csv_file_path, chose, user_chose=2) # Tạo prompt cho người dùng 
    gemini_response = get_gemini_response(prompt) # Lấy kết quả từ model
    write_csv_pandas(gemini_response, output_file) # Ghi kết quả vào file CSV

    print(f"File CSV đã được sắp xếp và lưu vào: {output_file}")

File CSV đã được sắp xếp và lưu vào: New_Quanlydonhang.csv
